# Hyperion Frequencies Dataset

Generating datasets for $v_c$, $v_m$ and $F_{v,max}$ in Adiabatic phase

In [1]:
import random
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from IPython.display import display, Latex

def generate_random_epsilon_B():
    return random.uniform(0.1, 10)
def generate_random_E_52():
    return random.uniform(1, 100)
def generate_random_n_1():
    return random.uniform(0.8, 1.2)
def generate_random_t_d():
    return 10**random.uniform(0, 2)
def generate_random_epsilon_E():
    return random.uniform(0.1,10)
def generate_random_D_28():
    return random.uniform(1,100)

epsilon_B_list = [generate_random_epsilon_B() for _ in range(100000)]
E_52_list = [generate_random_E_52() for _ in range(100000)]
n_1_list = [generate_random_n_1() for _ in range(100000)]
t_d_list = [generate_random_t_d() for _ in range(100000)]
epsilon_E_list = [generate_random_epsilon_E() for _ in range(100000)]
D_28_list = [generate_random_D_28() for _ in range(100000)]
v_c_list = []
v_m_list = []
Fv_max_list = []
a = (2.7) * (10**12)
b = (5.7) * (10**14)
c = (1.1) * (10**5)
for i in range(10000):
    #equation of v_c
    val = (a) * ((epsilon_B_list[i])**(-1.5)) * ((E_52_list[i]*t_d_list[i])**(-0.5)) / n_1_list[i]
    v_c_list.append(val)
    #equation of v_m
    val = (b) * ((epsilon_B_list[i])**(0.5)) * ((epsilon_E_list[i])**2) * ((E_52_list[i])**(0.5)) * ((t_d_list[i])**(-1.5))
    v_m_list.append(val)
    #equation of F_(v,max)
    val = (c) * ((epsilon_B_list[i])**(0.5)) * (E_52_list[i]) * ((n_1_list[i])**(0.5)) * ((D_28_list[i])**(-2))
    Fv_max_list.append(val)

noise_factor_v_c = 0.0001
noise_factor_v_m = 0.001
noise_factor_Fv_max = 0.0001
#adding noise
for i in range(4000):
    v_c_list[round(random.uniform(0,10000-1))]+= np.random.normal(0, noise_factor_v_c)
    v_m_list[round(random.uniform(0,10000-1))]+=np.random.normal(0, noise_factor_v_m)
    Fv_max_list[round(random.uniform(0,10000-1))]+=np.random.normal(0, noise_factor_Fv_max)


Create CSV files for the above datasets (adiabatic)

In [2]:
import csv

with open('adiabatic_dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    list1 = [['v_c','v_m','F_vmax','epsilon_B','epsilon_E','E_52','t_d','n_1','D_28']]
    for i in range(10000):
        temp_list=[]
        temp_list.append(v_c_list[i])
        temp_list.append(v_m_list[i])
        temp_list.append(Fv_max_list[i])
        temp_list.append(epsilon_B_list[i])
        temp_list.append(epsilon_E_list[i])
        temp_list.append(E_52_list[i])
        temp_list.append(t_d_list[i])
        temp_list.append(n_1_list[i])
        temp_list.append(D_28_list[i])
        list1.append(temp_list)
    writer.writerows(list1)


Running multiple linear regression to get relation between parameters

Step 1 : Create logarithmic variants of the lists

In [3]:
log_epsilon_B_list = []
log_E_52_list = []
log_n_1_list = []
log_t_d_list = []
log_epsilon_E_list = []
log_D_28_list = []
log_v_c_list = []
log_v_m_list = []
log_Fv_max_list = []
for i in range(10000):
    log_epsilon_B_list.append(math.log(epsilon_B_list[i],10))
    log_E_52_list.append(math.log(E_52_list[i],10))
    log_n_1_list.append(math.log(n_1_list[i],10))
    log_t_d_list.append(math.log(t_d_list[i],10))
    log_D_28_list.append(math.log(D_28_list[i],10))
    log_epsilon_E_list.append(math.log(epsilon_E_list[i],10))
    log_v_c_list.append(math.log(abs(v_c_list[i]),10))
    log_v_m_list.append(math.log(abs(v_m_list[i]),10))
    log_Fv_max_list.append(math.log(Fv_max_list[i],10))



Step 2 : Run multiple linear regression on the logarithmic lists obtained above

Step 2.1 : For $v_c$

In [4]:


# Combine the input features into a single array
X1 = np.column_stack((log_E_52_list, log_epsilon_B_list, log_n_1_list, log_t_d_list))

# Convert log_v_c_list to a NumPy array
y1 = np.array(log_v_c_list)

# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Create a linear regression model
model1 = LinearRegression()

# Train the model on the training data
model1.fit(X_train1, y_train1)

# Make predictions on the testing data
y_pred1 = model1.predict(X_test1)

# Print the coefficients and intercept
mse = mean_squared_error(y_test1, y_pred1)
print('Coefficients:', model1.coef_)
print('Intercept:', model1.intercept_)
print('Error: ',mse)
display(Latex(f'${{v_c}} = {10**model1.intercept_} * E_{{52}}^{({round(model1.coef_[0],2)})} * \epsilon_B^{({round(model1.coef_[1],2)})} * n_1^{({round(model1.coef_[2],2)})} * t_d^{({round(model1.coef_[3],2)})} $'))

Coefficients: [-0.5 -1.5 -1.  -0.5]
Intercept: 12.431363764158977
Error:  8.409099477474942e-29


<IPython.core.display.Latex object>

Step 2.2 : For $v_m$

In [5]:
# Combine the input features into a single array
X2 = np.column_stack((log_E_52_list, log_epsilon_B_list, log_epsilon_E_list, log_t_d_list))

# Convert log_v_c_list to a NumPy array
y2 = np.array(log_v_m_list)

# Split the data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Create a linear regression model
model2 = LinearRegression()

# Train the model on the training data
model2.fit(X_train2, y_train2)

# Make predictions on the testing data
y_pred2 = model2.predict(X_test2)

# Print the coefficients and intercept
print('Coefficients:', model2.coef_)
print('Intercept:', model2.intercept_)

display(Latex(f'${{v_m}} = {(10**model2.intercept_)} * E_{{52}}^{({round(model1.coef_[0],2)})} * \epsilon_B^{({round(model2.coef_[1],2)})} * \epsilon_e^{({round(model2.coef_[2],2)})} * t_d^{({round(model2.coef_[3],2)})} $'))

Coefficients: [ 0.5  0.5  2.  -1.5]
Intercept: 14.755874855672484


<IPython.core.display.Latex object>

Step 2.3 : For $F_{v,max}$

In [6]:
# Combine the input features into a single array
X3 = np.column_stack((log_E_52_list, log_epsilon_B_list, log_n_1_list, log_D_28_list))

# Convert log_v_c_list to a NumPy array
y3 = np.array(log_Fv_max_list)

# Split the data into training and testing sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Create a linear regression model
model3 = LinearRegression()

# Train the model on the training data
model3.fit(X_train3, y_train3)

# Make predictions on the testing data
y_pred3 = model3.predict(X_test3)

# Print the coefficients and intercept
print('Coefficients:', model3.coef_)
print('Intercept:', model3.intercept_)

display(Latex(f'${{v_c}} = {10**model3.intercept_} * E_{{52}}^{({round(model3.coef_[0],2)})} * \epsilon_B^{({round(model3.coef_[1],2)})} * n_1^{({round(model3.coef_[2],2)})} * D_{{28}}^{({round(model3.coef_[3],2)})} $'))

Coefficients: [ 1.          0.5         0.50000001 -2.        ]
Intercept: 5.0413926822408035


<IPython.core.display.Latex object>

----------------------------------------------------------------------------------------------------------------------------------------------------------------------

Generating datasets for $v_c$, $v_m$ and $F_{v,max}$ in early radiative phase

In [7]:
v_c_rad_list = []
v_m_rad_list = []
Fv_max_rad_list = []
a_rad = 1.3 * (10**13)
b_rad = 1.2 * (10**14)
c_rad = 4.5 * (10**3)
lorentz_factor = []
k = 1
for i in range(10000):
    lorentz_factor.append(random.uniform(100,1000))
    val = (a_rad) * ((epsilon_B_list[i])**(-1.5)) * ((E_52_list[i])**(-4/7)) * ((n_1_list[i])**(-13/14)) * ((t_d_list[i])**(-2/7)) * (lorentz_factor[i]**(4/7))
    v_c_rad_list.append(val)
    val = (b_rad) * ((epsilon_B_list[i])**(0.5)) * ((epsilon_E_list[i])**(2)) * ((E_52_list[i])**(4/7)) * ((n_1_list[i])**(-1/14)) * ((t_d_list[i])**(-12/7)) * (lorentz_factor[i]**(-4/7))
    v_m_rad_list.append(val)
    val = (a_rad) * ((epsilon_B_list[i])**(0.5)) * ((E_52_list[i])**(8/7)) * ((n_1_list[i])**(5/14)) * (D_28_list[i]**(-2)) * ((t_d_list[i])**(-3/7)) * (lorentz_factor[i]**(-8/7))
    Fv_max_rad_list.append(val)

for i in range(5000):
    v_c_rad_list[round(random.uniform(0,10000-1))]+= np.random.normal(0, noise_factor_v_c)
    v_m_rad_list[round(random.uniform(0,10000-1))]+=np.random.normal(0, noise_factor_v_m)
    Fv_max_rad_list[round(random.uniform(0,10000-1))]+=np.random.normal(0, noise_factor_Fv_max)


Generate CSV files for above datasets (radiative)

In [8]:
with open('radiative_dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    list1 = [['v_c','v_m','F_vmax','epsilon_B','epsilon_E','E_52','t_d','n_1','D_28','lorentz_factor']]
    for i in range(10000):
        temp_list=[]
        temp_list.append(v_c_rad_list[i])
        temp_list.append(v_m_rad_list[i])
        temp_list.append(Fv_max_rad_list[i])
        temp_list.append(epsilon_B_list[i])
        temp_list.append(epsilon_E_list[i])
        temp_list.append(E_52_list[i])
        temp_list.append(t_d_list[i])
        temp_list.append(n_1_list[i])
        temp_list.append(D_28_list[i])
        temp_list.append(lorentz_factor[i])
        list1.append(temp_list)
    writer.writerows(list1)



In [9]:
log_v_c_rad_list=[]
log_v_m_rad_list=[]
log_Fv_max_rad_list=[]
log_lorentz_factor=[]
for i in range(10000):
    log_lorentz_factor.append(math.log(lorentz_factor[i],10))
    log_v_c_rad_list.append(math.log(abs(v_c_rad_list[i]),10))
    log_v_m_rad_list.append(math.log(abs(v_m_rad_list[i]),10))
    log_Fv_max_rad_list.append(math.log(Fv_max_rad_list[i],10))

In [10]:
# Combine the input features into a single array
X = np.column_stack((log_E_52_list, log_epsilon_B_list, log_n_1_list, log_t_d_list,log_lorentz_factor))

# Convert log_v_c_list to a NumPy array
y = np.array(log_v_c_rad_list)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Print the coefficients and intercept
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)

Coefficients: [-0.57142857 -1.5        -0.92857143 -0.28571429  0.57142857]
Intercept: 13.113943352306821
